In [5]:
# Для каждого слова -- кроме самых частотных, служебных, числительных и именованных сущностей
# находим первую и последнюю дату упоминания.
import pickle
import pymorphy2
import pandas as pd
import json
from ner.utils import md5_hashsum, download_untar
from glob import glob
from datetime import date
from ner.network import NER
from ner.corpus import Corpus
from nltk.corpus import stopwords

In [9]:
with open("texts.pkl", "rb") as file:
    data = pickle.load(file)

In [10]:
data = data.reset_index(drop=True)

In [11]:
data.head()

,source,author,text,date
0,Lenta.ru,Unknown,Вопреки апокалиптическим прогнозам американски...,2000-01-01
1,Интерфакс,Unknown,Мадрид не выдаст Венесуэле укрывшегося в посо...,2000-01-01
2,Интерфакс,Unknown,Пожар на складе резины произошел в промзоне Н...,2000-01-01
3,Интерфакс,Unknown,Полиция Нью-Йорка обвинила трех подростков в ...,2000-01-01
4,Интерфакс,Unknown,Трамп направил кандидатуру постпреда США при ...,2000-01-01


In [12]:
with open("most_frequent.txt", 'r') as file:
    most_freq = [word for word in file.read().split('\n')]

In [21]:
morph = pymorphy2.MorphAnalyzer()
stopwords = set(stopwords('russian'))

def tokenize(text):
    return ([word.strip('.,:;?!()""''') for word in text.split(" ")])

def lemmatize(text):
    return ([morph.parse(word)[0].normal_form for word in tokenize(text)])

TypeError: 'set' object is not callable

In [15]:
with open('model/params.json') as f:
    network_params = json.load(f)

corpus = Corpus(dicts_filepath='model/dict.txt')

network = NER(corpus, verbouse=False, pretrained_model_filepath='model/ner_model', **network_params)

INFO:tensorflow:Restoring parameters from model/ner_model


In [16]:
def is_ok(word):
    if word in stopwords:
        return (False)
    if word in most_freq:
        return (False)
    if 'NUMR' in morph.parse(word)[0].tag:
        return (False)
    return (True)

In [17]:
text = lemmatize(data["text"][0])

In [18]:
tags = network.predict_for_token_batch([text])[0]

In [19]:
for word, tag in zip(text, tags):
    print(word, tag)

в O
о O
п O
р O
е O
к O
и O
  O
а O
п O
о O
к O
а O
л O
и O
п O
т O
и O
ч O
е O
с O
к O
и O
й O
  O
п O
р O
о O
г O
н O
о O
з O
  O
а O
м O
е O
р O
и O
к O
а O
н O
с O
к O
и O
й O
  O
к O
о O
м O
п O
ь O
ю O
т O
е O
р O
н O
ы O
й O
  O
с O
п O
е O
ц O
и O
а O
л O
и O
с O
т O
  O
н O
о B-PER
в O
о O
г O
о O
д O
н O
и O
й O
  O
н O
о B-PER
ч O
ь O
  O
н O
а O
  O
1 O
  O
я O
н O
в O
а O
р O
ь O
  O
2 O
0 O
0 O
0 O
  O
г O
о O
д O
  O
н O
е B-PER
  O
п O
р O
и O
н O
е O
с O
т O
и O
  O
н O
и O
к O
а O
к O
о O
й O
  O
к O
а O
т O
а O
с O
т O
р O
о O
ф O
а O
  O
- O
  O
н O
и O
  O
г O
л O
о O
б O
а O
л O
ь O
н O
ы O
й O
  O
н O
и O
  O
л O
о O
к O
а O
л O
ь O
н O
ы B-PER
й I-PER
  O
в B-PER
е O
с O
ь O
  O
к O
р O
и O
т O
и O
ч O
е O
с O
к O
и O
й O
  O
с O
и O
с O
т O
е O
м O
а O
  O
ж O
и O
з O
н O
е O
о B-ORG
б I-ORG
е I-ORG
с I-ORG
п I-ORG
е I-ORG
ч I-ORG
е I-ORG
н I-ORG
и I-ORG
е I-ORG
  I-ORG
п I-ORG
е I-ORG
р I-ORG
е I-ORG
н I-ORG
е O
с O
т O
и O
  O
р O
о O
к O
о O
в O
о O
й O
  O


In [ ]:
reverse_index = {}

In [ ]:
for i in range(len(data["text"])):
    if i % 100 == 0:
        with open("reverse_index.pkl", "wb") as file:
            pickle.dump(reverse_index, file)
    date = data["date"][i]
    text = lemmatize(data["text"][i])
    tags = network.predict_for_token_batch([text])[0]
    for word, tag in zip(text, tags):
        if tag == 'O' and is_ok(word):
            if word not in reverse_index:
                